In [1]:
from ppanlp import *

In [2]:
mergeddf=pd.read_csv('data/data.metadata.merged.ECCOII.csv', dtype=str)
# mergeddf
row = mergeddf.iloc[0]
dict(row)

{'id': '6362',
 'source_id': 'CB0131343042',
 'record_id': 'N20236',
 'title': 'A COMPENDIOUS English Grammar.',
 'subtitle': nan,
 'sort_title': 'COMPENDIOUS English Grammar.',
 'author': 'Pardon, William',
 'item_type': 'Excerpt',
 'book_journal': "A new general English dictionary; peculiarly calculated for the use and improvement of such as are unacquainted with the learned languages. Wherein the difficult words, and technical terms made use of in anatomy, architecture, arithmetick, algebra, astronomy, botany, chymistry, divinity, gardening, grammar, hawking, heraldry, history, horsemanship, hunting, husbandry, law, logick, mathematicks, mechanicks, milit. affairs, musick, navigation, painting, poetry, rhetorick, sculpture, surgery, &c. are not only fully explain'd, but accented on their proper syllables, to prevent a vicious pronunciation; and mark'd with initial letters, to denote the part of speech, to which each word peculiarly belongs. To which is prefixed, a compendious Englis

In [3]:
cache = get_page_cache()
eg_id = random.choice(list(cache.keys()))

In [4]:
eg_item = random.choice(list(cache[eg_id].items()))
eg_item

('00430',
 {'pageid': '00430',
  'assetid': '3327594641',
  'ocrlanguage': 'English',
  'ocr': 82.0,
  'imagelink': '149390110000430.TIF',
  'text': "\n\t\t\t( 42 ) So Misers are like Tavern-Soakers, WSill drink and swill like any Stoakers: Still ever s~ure to drink the most, Provided it is on Free-coft : And when so veryT drunk they get, They; own at least two Plumbs Effate. Gorg'd with their Gold as Men with Liquor,.i They- fqueamith grow, and plag~uy sick Sir: Then tak~e a Nap to lull their Pain - Their Heir fireight fpews it up again. M-uch M~oneyT had our 'Squire spent, ~And much by his fame Method went. At every publick Place he play'd, But chiefly at the Mlafquerade : Wiitnefs the scurvy· natty Trick, WVhen he Ten Thousand loss on Tick, By· wLhich he wans much mortified, And for- three Mitonthis wans f~orc'd to hide. As many more before have done, So that he's not the' onIy one. Till M~arriage set the MIatter right, And refcu'd him from dismal Plight. N~4ow fond again hiis Luck~

In [5]:
# Function to get pages for text, using sqlitedict cache made from ECCO page XMLs

def get_pages_row(row):
    textid=row.psmid
    textpages = cache.get(textid)
    if not textpages: return []
    
    opages = []
    if '-' in row.pages_digital:
        start,end=row.pages_digital.split('-',1)
        start,end=int(start),int(end)
        for page_num in range(start,end+1):
            page_id = f'{page_num:04}0'
            page = textpages.get(page_id)
            if page:
                opages.append(page)
    else:
        opages = list(textpages.values())
    
    for d in opages:
        d['docid']=row.docid

    return opages

In [14]:
# Get all pages
all_pages = [page for i,row in mergeddf.iterrows() for page in get_pages_row(row)]
pages_df = pd.DataFrame(all_pages, dtype=str).set_index('docid')
pages_df

,pageid,assetid,ocrlanguage,ocr,imagelink,text
docid,,,,,,
CB0131343042,00060,3331343047,English,92.22,171420150000060.TIF,\n\t\t\tI - - A C OM P EN ·IOUS English Gramma...
CB0131343042,00070,3331343048,English,90.67,171420150000070.TIF,\n\t\t\tthere four Parts by the received Names...
CB0131343042,00080,3331343049,English,86.44,171420150000080.TIF,\n\t\t\tNotice thall be taken hereafter. The t...
CB0131343042,00090,3331343050,English,85.11,171420150000090.TIF,"\n\t\t\terpf as, a Horse, a Man, a Tree, an Im..."
CB0131343042,00100,3331343051,English,82.22,171420150000100.TIF,"\n\t\t\tblne; jbe hers, are. for the Feminine;..."
...,...,...,...,...,...,...
CB0129230262,00190,3329230280,English,68.67,144240030000190.TIF,"\n\t\t\tAd~J a growing Mlit'chief,. while a th..."
CB0129230262,00200,3329230281,English,81.0,144240030000200.TIF,\n\t\t\tthere is a Shadow of thlis Pra.aice in...
CB0129230262,00210,3329230282,English,79.44,144240030000210.TIF,"\n\t\t\tEar, anld abuses the trure Relish, eve..."


In [22]:
# Merge back in
pages_and_metadata = pages_df.merge(mergeddf, on='docid', suffixes=('_page','_doc')).reset_index().set_index(['docid','pageid']).drop('index',axis=1)
pages_and_metadata

assetid_page ocrlanguage ocr_page            imagelink  \
docid        pageid                                                          
CB0131343042 00060    3331343047     English    92.22  171420150000060.TIF   
             00070    3331343048     English    90.67  171420150000070.TIF   
             00080    3331343049     English    86.44  171420150000080.TIF   
             00090    3331343050     English    85.11  171420150000090.TIF   
             00100    3331343051     English    82.22  171420150000100.TIF   
...                          ...         ...      ...                  ...   
CB0129230262 00190    3329230280     English    68.67  144240030000190.TIF   
             00200    3329230281     English     81.0  144240030000200.TIF   
             00210    3329230282     English    79.44  144240030000210.TIF   
             00220    3329230283     English    80.22  144240030000220.TIF   
             00230    3329230284     English    72.56  144240030000230.TIF   

                                                                  text    id  \
docid        pageid                                                            
CB0131343042 00060   \n\t\t\tI - - A C OM P EN ·IOUS English Gramma...  6362   
             00070   \n\t\t\tthere four Parts by the received Names...  6362   
             00080   \n\t\t\tNotice thall be taken hereafter. The t...  6362   
             00090   \n\t\t\terpf as, a Horse, a Man, a Tree, an Im...  6362   
             00100   \n\t\t\tblne; jbe hers, are. for the Feminine;...  6362   
...                                                                ...   ...   
CB0129230262 00190   \n\t\t\tAd~J a growing Mlit'chief,. while a th...  6428   
             00200   \n\t\t\tthere is a Shadow of thlis Pra.aice in...  6428   
             00210   \n\t\t\tEar, anld abuses the trure Relish, eve...  6428   
             00220   \n\t\t\t:W~ide D~e'e~As and equal Superfluit~i...  6428   
             00230   \n\t\t\t....g~ualem nequeo -monfrare, Ed fenti...  6428   

                        source_id record_id                           title  \
docid        pageid                                                           
CB0131343042 00060   CB0131343042    N20236  A COMPENDIOUS English Grammar.   
             00070   CB0131343042    N20236  A COMPENDIOUS English Grammar.   
             00080   CB0131343042    N20236  A COMPENDIOUS English Grammar.   
             00090   CB0131343042    N20236  A COMPENDIOUS English Grammar.   
             00100   CB0131343042    N20236  A COMPENDIOUS English Grammar.   
...                           ...       ...                             ...   
CB0129230262 00190   CB0129230262    W38526                    THE PREFACE.   
             00200   CB0129230262    W38526                    THE PREFACE.   
             00210   CB0129230262    W38526                    THE PREFACE.   
             00220   CB0129230262    W38526                    THE PREFACE.   
             00230   CB0129230262    W38526                    THE PREFACE.   

                    subtitle                    sort_title           author  \
docid        pageid                                                           
CB0131343042 00060       NaN  COMPENDIOUS English Grammar.  Pardon, William   
             00070       NaN  COMPENDIOUS English Grammar.  Pardon, William   
             00080       NaN  COMPENDIOUS English Grammar.  Pardon, William   
             00090       NaN  COMPENDIOUS English Grammar.  Pardon, William   
             00100       NaN  COMPENDIOUS English Grammar.  Pardon, William   
...                      ...                           ...              ...   
CB0129230262 00190       NaN                      PREFACE.     Watts, Isaac   
             00200       NaN                      PREFACE.     Watts, Isaac   
             00210       NaN                      PREFACE.     Watts, Isaac   
             00220       NaN                      PREFACE.     Watt

In [24]:
pages_and_metadata.to_csv('data/data.pages_and_metadata.csv')

In [13]:
mergeddf.docid

0     CB0131343042
1     CB0129438775
2     CB0128108681
3     CB0129206667
4     CB0129206667
5     CB0131116162
6     CB0132627374
7     CB0132627374
8     CB0132796469
9     CB0129122950
10    CB0126124028
11    CB0131333744
12    CB0131333744
13    CB0129436673
14    CB0129436673
15    CB0127376362
16    CB0127376362
17    CB0131341019
18    CB0127060264
19    CB0128330490
20    CB0132422117
21    CB0127251015
22    CB0129230262
Name: docid, dtype: object

In [ ]:
pages_df.